# Parent Document Retriever

ParentDocumentRetriever 是 LangChain 中一个用于文档检索的工具，旨在解决文档分割和检索之间的平衡问题。它通过存储小文档块的嵌入，并在检索时先找到相似的小块，然后返回对应的完整父文档来实现更准确和全面的检索。

**工作原理**

* **存储小文档块的嵌入**： 将文档分割成更小的块（chunk），并为每个块生成嵌入向量。这些嵌入向量捕捉了小块的语义信息。
* **检索时先找相似小块**： 当用户进行检索时，ParentDocumentRetriever 首先根据用户query的嵌入向量，在存储的小文档块的嵌入向量中找到最相似的几个小块。
* **然后返回对应的完整父文档**： 找到相似的小块后，ParentDocumentRetriever 会返回这些小块所属的完整父文档。



**核心优势**

* **语义准确**： 小块生成更精确的语义嵌入，提高匹配度。
* **上下文保留**： 返回完整父文档，包含充分上下文信息。
* **灵活高效**： 支持自定义分割策略，小块检索更快速。

**主要不足**

* **存储开销大**： 需同时存储小块和父文档，占用空间多。
* **性能有损**： 两阶段检索增加延迟，可能略慢。
* **配置复杂**： 需要专业知识，分割策略选择有技巧。

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryByteStore
from langchain_ollama import OllamaEmbeddings
from langchain.retrievers import ParentDocumentRetriever

from chat_model_client import get_model

## 1.声明向量存储和普通存储

In [ ]:
embeddings = OllamaEmbeddings(model = "llama2-chinese")
## 向量数据库用来存储小块
vectorstore = Chroma(
    collection_name="parent_documents",
    embedding_function=embeddings
)
## store用来存储原始文档
store = InMemoryByteStore()

## 2. 构建Retriever

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
retriever = ParentDocumentRetriever(
    vectorstore = vectorstore,
    docstore = store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

## 3.添加文档

In [8]:
loader = WebBaseLoader([
    "https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a",
    "https://cn.chinadaily.com.cn/a/202412/04/WS67502215a310b59111da71ab.html"
])
documents = loader.load()
retriever.add_documents(documents)


## 4.检索问题

In [9]:
retriever.invoke("马拉松经济价值")

[Document(metadata={'source': 'https://cn.chinadaily.com.cn/a/202412/04/WS67502215a310b59111da71ab.html', 'title': '算一算马拉松热背后的经济账 - 中国日报网', 'description': '中国日报网', 'language': 'No language found.'}, page_content='11月3日，参赛选手奔跑在长安街上。当日，2024北京马拉松赛暨全国马拉松锦标赛（第3站）鸣枪开跑。新华社记者 张晨霖 摄\n马拉松比赛蕴含的巨大消费空间，让各方想尽办法参与其中。但也有跑友反映，随着举办城市增多、市场下沉，同质化严重、组织混乱等一系列问题逐渐显现。\n一场马拉松赛事，举办日不过只有一天，却是一项“超级大工程”。\n业内人士称，举办一场大型马拉松赛事往往考验主办方的动员能力、专业组织能力、多部门协调配合能力，甚至可以体现一个城市的综合治理能力。\n“我们将其定义为一个‘巨系统’，它是由内部无数件小事组成的‘子系统’，经过资源优化配置综合呈现的成果。”李长征说。\n“大型赛事极大考验迸发式城市消化力，比如医疗、安保、交通等。”中国田径协会执委、马拉松赛事专家张思杰表示，尊重比赛专业性、保障办赛安全是基础。\n此外，马拉松赛事要想走得更远，做得更好，清晰的品牌规划十分重要。张思杰认为，除了赛事期间旅游消费、宣传等算得出来的经济价值外，有能力的大城市或品牌赛事还应顺应大形势和城市发展规划，承载“平台和工具”的功能，结合经济建设需求做叠加，例如与地方产业、人才引进等方面做结合。\n谈到赛事未来增长点，张思杰表示，我国的马拉松完赛人口仍较少，随着人们对运动健康的诉求加大，不同类别形式、长短距离的细分赛道，景区、园区赛事等仍有增长空间。\n\n\n\n\n\n\n                【责任编辑：富文佳】\n              \n\nCOMPO\nWS67502215a310b59111da71ab\nhttps://cn.chinadaily.com.cn/a/202412/04/WS67502215a310b59111da71ab.html\n\n\n \n\n\n\n\n\n\n\n专题\n\n\n\n\n\n新春走基